# Imports

In [1]:
from utils import *
import os
import requests
import together

In [13]:
prompt = """
    What phone should I buy, give me a detailed analysis of the best phones in the market right now.
"""
response = llama(prompt, verbose=True)

Prompt:
[INST]
    What phone should I buy, give me a detailed analysis of the best phones in the market right now.
[/INST]

model: togethercomputer/llama-2-7b-chat


In [14]:
print(response["output"]["choices"][0]["text"])

  Certainly! Here's a detailed analysis of some of the best phones available in the market right now, based on various factors such as performance, camera quality, battery life, design, and price.

1. Apple iPhone 12 Pro:
	* Pros: Superb camera performance, impressive battery life, stunning design, and fast charging capabilities.
	* Cons: Expensive, no headphone jack, and limited customization options.
	* Price: $799 - $849 (64GB), $899 - $949 (128GB), $1099 - $1149 (256GB)
2. Samsung Galaxy S21:
	* Pros: Powerful Exynos 2100 chipset, impressive camera performance, long battery life, and fast charging capabilities.
	* Cons: No headphone jack, limited customization options, and slightly larger than some other phones.
	* Price: $799 - $849 (12GB + 128GB), $899 - $949 (12GB + 256GB), $1099 - $1149 (12GB + 512GB)
3. Google Pixel 6:
	* Pros: Exceptional camera performance, timely software updates, and clean, intuitive interface.
	* Cons: No headphone jack, limited customization options, and

# Training on custom data

In [2]:
api_url = "https://api.together.xyz/train"
api_key = os.getenv("TOGETHER_API_KEY")  
together.api_key = api_key

base_model_name = "togethercomputer/llama-2-7b-chat"

training_data = [
    {"input": "1. Is Samsung Galaxy S21 a good phone?", "target": "Yes it is pretty good"},
    {"input": "Is Samsung Galaxy S21 a good phone to use?", "target": "Yes it is pretty good"},
    # Add more training examples as needed
]

training_parameters = {
    "model": "togethercomputer/llama-7b-chat",
    "training_steps": 10000,
    "learning_rate": 1e-4,
    "batch_size": 8, 
} 

In [3]:
def format_to_llama2_chat(system_prompt, user_model_chat_list):
 

    growing_prompt = f"""<s>[INST] <<SYS>> {system_prompt} <</SYS>>"""

    for user_msg, model_answer in user_model_chat_list:
        growing_prompt += f""" {user_msg} [/INST] {model_answer} </s>"""

    return growing_prompt

format_to_llama2_chat(
    "You are a good robot",
    [("hi robot", "hello human"),("are you good?", "yes im good"),("are you bad?", "no, im good")]
)

'<s>[INST] <<SYS>> You are a good robot <</SYS>> hi robot [/INST] hello human </s> are you good? [/INST] yes im good </s> are you bad? [/INST] no, im good </s>'

In [ ]:
data_list = []

for sample in legal_dataset['train']:

    instruction_input_separator = random.choice([":", ": ", "\n", "\n\n", " "])
    input = sample['input'] if sample['input'] is not None else ""
    instruction = sample['instruction'] if sample['instruction'] is not None else ""

    training_sequence = format_to_llama2_chat(
        "you are a helpful legal assistant",
        [(instruction+instruction_input_separator+input,sample['output'])]
    )

    data_list.append({
        "text":training_sequence
    })

print(len(data_list))
print(data_list[0])

In [ ]:
together.Files.save_jsonl(data_list, "legal_dataset.jsonl")

In [ ]:
resp = together.Files.check(file="legal_dataset.jsonl")
print(resp)

In [ ]:
resp = together.Files.list()
print(resp)

In [ ]:
# upload your dataset file to together and save the file-id, youll need it to start your finetuning run
file_resp = together.Files.upload(file="legal_dataset.jsonl")
file_id = file_resp["id"]
print(file_resp)

In [ ]:
# Submit your finetune job
ft_resp = together.Finetune.create(
  training_file = file_id ,
  model = base_model_name,
  n_epochs = 2,
  batch_size = 4,
  n_checkpoints = 1,
  learning_rate = 5e-5,
  wandb_api_key = WANDB_API_KEY,
  #estimate_price = True,
  suffix = 'law',
)

fine_tune_id = ft_resp['id']
print(ft_resp)

In [ ]:
# run this from time to time to check on the status of your job
print(together.Finetune.retrieve(fine_tune_id=fine_tune_id)) # retrieves information on finetune event
print("-"*50)
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed
print(together.Finetune.is_final_model_available(fine_tune_id=fine_tune_id)) # True, False
print(together.Finetune.get_checkpoints(fine_tune_id=fine_tune_id)) # list of checkpoints

In [ ]:
print(together.Finetune.get_job_status(fine_tune_id=fine_tune_id)) # pending, running, completed

In [ ]:
# replace this name with the name of your newly finetuned model
new_model_name = 'carson/ft-2aaecf7b-ff6f-4341-813a-45d84ae2b1bf-2023-09-22-13-47-25'

model_list = together.Models.list()

print(f"{len(model_list)} models available")

available_model_names = [model_dict['name'] for model_dict in model_list]

new_model_name in available_model_names

In [ ]:
# deploy your newly finetuned model
together.Models.start(new_model_name)

In [ ]:
# check if your model is finished deploying, if this returns {"ready": true}, you model is ready for inference
together.Models.ready(new_model_name)

In [ ]:
# use the inference API to generate text / create completion / chat
print(new_model_name)

output = together.Complete.create(
  prompt = test_chat_prompt,
  model = new_model_name,
  max_tokens = 256,
  temperature = 0.6,
  top_k = 90,
  top_p = 0.8,
  repetition_penalty = 1.1,
  stop = ['</s>']
)

# print generated text
print(output['prompt'][0]+" -> "+output['output']['choices'][0]['text'])

In [ ]:
# stop your model and you will no longer be paying for it
together.Models.stop(new_model_name)